To do: Convert this to a class

In [1]:
import networkx as nx

In [2]:
vocab_G = nx.read_gml('./Graphs/corpus_vocab.gml')
len(vocab_G)

11553

In [3]:
def k_most_common_from(G: nx.DiGraph, target, k = 10, occur = 'count') -> tuple:
    """Find the k most common words after a target word
    Parameters:
        G (nx.DiGraph): a directed network
        target (str): the target word
        k (int): the limit (default: 10), if None then all are retrieved
    Returns:
        tuple: next words and their probabilities"""
    
    next_words = {i[1]: i[2][occur] for i in G.out_edges(target, data = True)}
    
    if len(next_words) > 0:
        total = sum(next_words.values())
        
        next_words = sorted({i: next_words[i] / total for i in next_words}.items(), key = lambda x: x[1], reverse = True)
        
        if k:
            return dict(next_words[: k])
        else:
            return dict(next_words)
    
    return dict()

In [4]:
k_most_common_from(vocab_G, '<s>', k = 10)

{'the': 0.12415469146238377,
 'i': 0.0944632290786137,
 'it': 0.04723161453930685,
 'but': 0.041631445477599324,
 'he': 0.034657650042265425,
 'we': 0.02345731191885038,
 'in': 0.02187235841081995,
 'at': 0.020181741335587488,
 'a': 0.01775147928994083,
 'and': 0.017645815722738798}